In [1]:
import logging
import pickle

from lenskit import util
from lenskit.algorithms import als
from lenskit import batch
from lenskit import topn

import pandas as pd
import numpy as np
from scipy import stats
import binpickle

from pytest import mark, approx

import lenskit.util.test as lktu
from lenskit.algorithms import Recommender
from lenskit.util import Stopwatch

from lenskit.batch import MultiEval
from lenskit.crossfold import partition_users, SampleN, SampleFrac
from lenskit.algorithms import basic, als
from lenskit.datasets import MovieLens
from lenskit import topn, util
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()

In [2]:
training_data = pd.read_parquet(r"/home/ac8826/final-project-team_pseudo_recommenders/Databases/Databases/cf_train.parquet/part-00003-f5f99d61-97a3-408a-9dcd-5d6fc378fff2-c000.snappy.parquet")

In [3]:
training_data.head()

,user_id,count,track_id,__index_level_0__,user_id_index,track_id_index
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,TRIQAUQ128F42435AD,0,103159,2994
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,TRIRLYL128F42539D1,1,103159,12575
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,2,TRMHBXZ128F4238406,2,103159,3524
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,TRYQMNI128F147C1C7,3,103159,32121
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,TRAHZNE128F9341B86,4,103159,20989


In [4]:
df_train = pd.DataFrame({'item':training_data['track_id'],'user':training_data['user_id']})

In [5]:
df_train_ratings = df_train.copy(deep = True)
df_train_ratings['rating'] = training_data['count']

In [6]:
df_train_ratings.head(3)

,item,user,rating
0,TRIQAUQ128F42435AD,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
1,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
2,TRMHBXZ128F4238406,b80344d063b5ccb3212f76538f3d9e43d87dca9e,2


In [7]:
Implicit_ALS_Model = als.ImplicitMF(250, iterations=15, reg = 0.1, weight = 100,method='cg')

In [8]:
fittable = util.clone(Implicit_ALS_Model)

In [9]:
fittable = Recommender.adapt(fittable)

In [10]:
import time
start = time.time()
fittable.fit(df_train_ratings)
end = time.time()
print(f'Time Taken to Fit The Data: {end - start}')

Time Taken to Fit The Data: 633.0114307403564


In [65]:
for idx, u in enumerate(tqdm(['a','b','c'])):
    print(f'idx: {idx}, u : {u}')
    if idx%1 == 0:
        print(idx)

  0%|          | 0/3 [00:00<?, ?it/s]

idx: 0, u : a
0
idx: 1, u : b
1
idx: 2, u : c
2


In [11]:
val_path = r'/home/ac8826/final-project-team_pseudo_recommenders/Databases/Databases/cf_validation.parquet/part-00000-09e59385-7dcd-4b76-801d-5178b66f79fb-c000.snappy.parquet'
val = pd.read_parquet(val_path)

In [12]:
val.head()

,user_id,count,track_id,__index_level_0__,user_id_index,track_id_index
0,0007140a3796e901f3190f12e9de6d7548d4ac4a,1,TRUFCYO128F422B898,0,700105,1036
1,0007140a3796e901f3190f12e9de6d7548d4ac4a,2,TROBZPR128F14808FF,1,700105,33556
2,0007140a3796e901f3190f12e9de6d7548d4ac4a,1,TRFGZUW128F92FC2AB,2,700105,37081
3,0007140a3796e901f3190f12e9de6d7548d4ac4a,1,TRHTCXG12903CC2F60,3,700105,44
4,0007140a3796e901f3190f12e9de6d7548d4ac4a,1,TRKFBTS128EF34E530,4,700105,13959


In [13]:
df_val = pd.DataFrame({'item':val['track_id'],'user':val['user_id']})

In [14]:
users = df_val.user.unique()

In [15]:
df_val_rating = df_val.copy(deep = True)
df_val_rating['rating'] = val['count']

In [17]:
recommendations_df = pd.DataFrame(columns = ['item', 'score', 'user', 'rank'])
ls_ndgc = []

for i, user in enumerate(tqdm(users)):
    ls_users = []
    ls_item = []
    ls_rank = []
    ls_score = []
    recs = fittable.recommend(user, 500)
    recs['user'] = user
    recs['rank'] = recs.groupby('user')['score'].rank(ascending=False)
    ls_users.extend(list(recs['user']))
    ls_item.extend(list(recs['item']))
    ls_rank.extend(list(recs['rank']))
    ls_score.extend(list(recs['score']))

    if True:
        recommendation_map = {'item': ls_item, 'user': ls_users, 'rank': ls_rank, 'score': ls_score}
        recommendations_df = pd.DataFrame(recommendation_map, columns= ['item', 'user', 'rank', 'score'])
        RecListAnalysisObj = topn.RecListAnalysis()
        RecListAnalysisObj.add_metric(topn.ndcg)
        results = RecListAnalysisObj.compute(recommendations_df, df_val[df_val['user']==user])
        ls_ndgc.extend(list(results['ndcg']))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [18]:
print("Recommendations Predictied")
print(f'NDGC Score: {sum(ndgc_list)/len(ndgc_list)}')

Recommendations Predictied
NDGC Score: 0.30407043768920755


In [4]:
for i in range(2):
    if True:
        print(1)

1
1
